In [2]:
import os
import re
import csv
import requests
import urllib.request
import pandas as pd
import numpy as np

from urllib.error import HTTPError
from bs4 import BeautifulSoup as bs

In [3]:
# csv preprocessisng for urls

In [4]:
mIds = pd.read_csv('./data/ml-20m/links.csv', encoding='utf-8')

In [5]:
mIds.drop(['tmdbId'], axis=1, inplace=True)

In [5]:
mIds['movieId'].isnull().sum() # Finding null

0

In [6]:
mIds['imdbId'].isnull().sum() # Finding null

0

In [7]:
def imdbIb_to_url(x):
    return 'tt'+str(x).zfill(7)

mIds['imdbId']= mIds['imdbId'].map(imdbIb_to_url)

In [8]:
imdb_ref = 'https://www.imdb.com/title/'
mIds['imdb_url'] = mIds['imdbId'].apply(lambda x: imdb_ref+x)

In [9]:
ml_ref = 'https://movielens.org/movies/'
mIds['ml_url'] = mIds['movieId'].apply(lambda x: ml_ref+str(x))

In [10]:
mIds.head(5)

,movieId,imdbId,tmdbId,imdb_url,ml_url
0,1,tt0114709,862.0,https://www.imdb.com/title/tt0114709,https://movielens.org/movies/1
1,2,tt0113497,8844.0,https://www.imdb.com/title/tt0113497,https://movielens.org/movies/2
2,3,tt0113228,15602.0,https://www.imdb.com/title/tt0113228,https://movielens.org/movies/3
3,4,tt0114885,31357.0,https://www.imdb.com/title/tt0114885,https://movielens.org/movies/4
4,5,tt0113041,11862.0,https://www.imdb.com/title/tt0113041,https://movielens.org/movies/5


In [11]:
mIds.tail(5)

,movieId,imdbId,tmdbId,imdb_url,ml_url
27273,131254,tt0466713,4436.0,https://www.imdb.com/title/tt0466713,https://movielens.org/movies/131254
27274,131256,tt0277703,9274.0,https://www.imdb.com/title/tt0277703,https://movielens.org/movies/131256
27275,131258,tt3485166,285213.0,https://www.imdb.com/title/tt3485166,https://movielens.org/movies/131258
27276,131260,tt0249110,32099.0,https://www.imdb.com/title/tt0249110,https://movielens.org/movies/131260
27277,131262,tt1724965,286971.0,https://www.imdb.com/title/tt1724965,https://movielens.org/movies/131262


In [12]:
# crawling through mIds

In [13]:
# Function for making directories
def makeDirs(path):
    try:
        os.makedirs(path)
    except OSError:
        if not os.path.isdir(path):
            raise

In [14]:
def crawlTitle(soup):
    title = soup.find('h1', attrs={'class':re.compile('^^TitleHeader')})    
    #title = soup.select_one('#__next > main > div > section.ipc-page-background.ipc-page-background--base.TitlePage__StyledPageBackground-wzlr49-0.dDUGgO > section > div:nth-child(4) > section > section > div.TitleBlock__Container-sc-1nlhx7j-0.hglRHk > div.TitleBlock__TitleContainer-sc-1nlhx7j-1.jxsVNt > h1')
    if title is not None:
        return title.get_text()
    return title

In [15]:
def crawlPlot(soup):
    plot = soup.find('span', attrs={'class':re.compile('^^GenresAndPlot')})  
    if plot is not None:
        return plot.get_text()
    return plot

In [16]:
def crawlPoster(soup):
    img_url = soup.find('a', attrs={'aria-label':'View {Title} Poster'})
    if img_url is None:
        return None
    img_url = 'https://www.imdb.com/' + re.search('href="(.+?)"',str(img_url)).group(1)
    raw = requests.get(img_url, headers = {"User-Agent" : "Mozilla/5.0"})
    _soup = bs(raw.content, 'html.parser')
    img_src = _soup.find_all('img')
    return img_src

In [17]:
# Plots data save at csv file
def savePlot(idx,title, plot):
    # write 
    with open(TXTPATH+'plots.csv', 'a+', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow([idx,title,plot])    

In [57]:
# Image download to dir
def savePoster(idx, poster):
    if poster is None:
        with open(TXTPATH+'no_poster.txt', 'a+', encoding='utf-8') as f:
            f.write(f'{idx}\n')
            return -1
    try:
        urllib.request.urlretrieve(poster[1]['src'], IMGPATH+str(idx).zfill(6)+'.jpg')
    except HTTPError as err:
        if err.code == 404: 
            try: urllib.request.urlretrieve(poster[2]['src'], IMGPATH+str(idx).zfill(6)+'.jpg')            
            except:
                with open(TXTPATH+'no_poster.txt', 'a+', encoding='utf-8') as f:
                    f.write(f'{idx}\n')
                    return -1           

In [61]:
IMGPATH = './data/posters/'
TXTPATH = './data/'

plots,posters = [], []
urls = mIds['imdb_url']

makeDirs(IMGPATH)

In [62]:
for i,url in enumerate(urls):
    raw = requests.get(url, headers = {"User-Agent" : "Mozilla/5.0"})
    soup = bs(raw.text, 'html.parser')
    title = crawlTitle(soup)
    plot = crawlPlot(soup)
    poster = crawlPoster(soup)
    # Check movieId
    m_id = mIds.loc[mIds['imdb_url'] == url]['movieId'].to_numpy()[0]
    print(i, m_id)
    # Download in dir
    savePlot(m_id,title, plot)
    savePoster(m_id, poster)

0 43708
1 43710


In [18]:
def crawlPlot(soup):
    plot = soup.find('li', attrs={'class':'ipl-zebra-list__item'})
    if plot is not None:
        return plot.get_text()
    return plot

In [36]:
def crawlTitle(soup):
    title = soup.find('a', attrs={'itemprop':'url'})    
    #title = soup.select_one('#__next > main > div > section.ipc-page-background.ipc-page-background--base.TitlePage__StyledPageBackground-wzlr49-0.dDUGgO > section > div:nth-child(4) > section > section > div.TitleBlock__Container-sc-1nlhx7j-0.hglRHk > div.TitleBlock__TitleContainer-sc-1nlhx7j-1.jxsVNt > h1')
    if title is not None:
        return title.get_text()
    return title

In [39]:
# Plots data save at csv file
def savePlot(idx,title, plot):
    # write 
    with open(TXTPATH+'plots_.csv', 'a+', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow([idx,title,plot])    

In [42]:
IMGPATH = './data/posters/'
TXTPATH = './data/'

plots,posters = [], []
urls = mIds['imdb_url'][15:16]

makeDirs(IMGPATH)

In [43]:
for i,url in enumerate(urls):
    url += '/plotsummary?ref_=tt_ov_pl'
    print(url)
    raw = requests.get(url, headers = {"User-Agent" : "Mozilla/5.0"})
    soup = bs(raw.text, 'html.parser')
    plot = crawlPlot(soup)
    title = crawlTitle(soup)
    # Check movieId
    url = url.replace('/plotsummary?ref_=tt_ov_pl','')
    print(url)
    m_id = mIds.loc[mIds['imdb_url'] == url]['movieId'].to_numpy()[0]
    print(i, m_id)
    # Download in dir
    print(title,plot)
    savePlot(m_id,title, plot)


https://www.imdb.com/title/tt0112641/plotsummary?ref_=tt_ov_pl
https://www.imdb.com/title/tt0112641
0 16
Casino 
A tale of greed, deception, money, power, and murder occur between two best friends: a mafia enforcer and a casino executive compete against each other over a gambling empire, and over a fast-living and fast-loving socialite.



https://www.imdb.com/title/tt0198781/plotsummary?ref_=tt_ov_pl